## predict

In [1]:
# UltralyticsのYOLOv8を使用して画像推論を実行
from ultralytics import YOLO
import os
from pathlib import Path
import glob

print("ライブラリをインポートしました")

# パスの設定
model_path = r"D:\prog_py\Yolo_trainer\runs\train\train9\weights\best.pt"
source_path = "detect_images"

# モデルファイルと画像フォルダの存在確認
if os.path.exists(model_path):
    print(f"✓ モデルファイル確認: {model_path}")
else:
    print(f"✗ モデルファイルが見つかりません: {model_path}")

if os.path.exists(source_path):
    images = [f for f in os.listdir(source_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"✓ 画像フォルダ確認: {source_path} ({len(images)}枚の画像)")
else:
    print(f"✗ 画像フォルダが見つかりません: {source_path}")

# YOLOモデルの読み込み
model = YOLO(model_path)
print(f"✓ モデルを読み込みました")
print(f"モデルクラス数: {len(model.names)}")
print(f"クラス名: {list(model.names.values())}")


ライブラリをインポートしました
✓ モデルファイル確認: D:\prog_py\Yolo_trainer\runs\train\train9\weights\best.pt
✓ 画像フォルダ確認: detect_images (13枚の画像)
✓ モデルを読み込みました
モデルクラス数: 1
クラス名: ['pod']


In [2]:
# 指定されたパラメータで画像推論を実行
print("推論を開始します...")

results = model.predict(
    source=source_path,      # 画像フォルダ: detect_images
    imgsz=1200,              # 画像サイズ: 800
    save_txt=True,          # テキストファイル保存: True
    line_width=1,           # バウンディングボックスの線の太さ: 1
    save=True,              # 推論結果の画像を保存
    conf=0.25,              # 信頼度のしきい値
    device='0',           # デバイス（GPUが利用可能な場合は'0'に変更）
    project='runs/detect',  # 結果保存ディレクトリ
    name='yolo_detection',  # 実験名 # 既存のディレクトリがある場合上書き
    retina_masks=True,
    agnostic_nms=True,
)

print("✓ 推論が完了しました！")
print(f"結果は runs/detect/yolo_detection に保存されました")


推論を開始します...

WARNING imgsz=[1200] must be multiple of max stride 32, updating to [1216]
image 1/13 d:\prog_py\yolo12_detect\detect_images\Image_000003.jpg: 1216x1120 78 pods, 95.4ms
image 2/13 d:\prog_py\yolo12_detect\detect_images\Image_000118.jpg: 1216x736 76 pods, 141.8ms
image 3/13 d:\prog_py\yolo12_detect\detect_images\Image_000188.jpg: 1216x704 58 pods, 82.0ms
image 4/13 d:\prog_py\yolo12_detect\detect_images\Image_000473.jpg: 1216x672 67 pods, 80.5ms
image 5/13 d:\prog_py\yolo12_detect\detect_images\Image_000565.jpg: 1216x512 80 pods, 81.6ms
image 6/13 d:\prog_py\yolo12_detect\detect_images\Image_000896.jpg: 1216x544 58 pods, 77.6ms
image 7/13 d:\prog_py\yolo12_detect\detect_images\Image_001075.jpg: 1216x896 43 pods, 93.9ms
image 8/13 d:\prog_py\yolo12_detect\detect_images\Image_001845.jpg: 1216x480 46 pods, 78.5ms
image 9/13 d:\prog_py\yolo12_detect\detect_images\Image_001851.jpg: 1216x608 17 pods, 77.0ms
image 10/13 d:\prog_py\yolo12_detect\detect_images\Image_001885.jpg: 1216

In [3]:
# dataset.yamlを使用したモデル検証
print("=== モデル検証を開始します ===\n")

# dataset.yamlのパス
dataset_yaml_path = r"D:\Data\pods\2025_yolo_pod\labels\YOLODataset\dataset.yaml"

# dataset.yamlファイルの存在確認
if os.path.exists(dataset_yaml_path):
    print(f"✓ データセット設定ファイル確認: {dataset_yaml_path}")
    
    # ファイルの内容を表示
    with open(dataset_yaml_path, 'r', encoding='utf-8') as f:
        content = f.read()
        print(f"\nデータセット設定:")
        print(content)
else:
    print(f"✗ データセット設定ファイルが見つかりません: {dataset_yaml_path}")
    print("検証を実行できません。")


=== モデル検証を開始します ===

✓ データセット設定ファイル確認: D:\Data\pods\2025_yolo_pod\labels\YOLODataset\dataset.yaml

データセット設定:
path: \\?\D:\Data\pods\2025_yolo_pod\labels\YOLODataset
train: images/train
val: images/val
test:

names:
    0: pod



In [ ]:
# 画像サイズ（imgsz）はmax stride 32の倍数でグリッドサーチ
if os.path.exists(dataset_yaml_path):
    print("🔍 画像サイズ（32の倍数, 1500まで）で検証を実行しています...")

    # 32の倍数で500から1504まで
    grid_imgsz = list(range(512, 1505, 32))

    for imgsz in grid_imgsz:
        print(f"\n--- 画像サイズ: {imgsz} で検証中 ---")
        validation_results = model.val(
            data=dataset_yaml_path,
            imgsz=imgsz,
            plots=True,
            batch=8,
            save_json=True,
            device='0',
            project='runs/val',
            name=f'val_imgs_{imgsz}',
            verbose=True,
        )
        metrics = getattr(validation_results, 'metrics', None)
        if metrics is not None:
            map_50_95 = getattr(metrics, 'mAP_0.5_0.95', None)
            precision = getattr(metrics, 'precision', None)
            recall = getattr(metrics, 'recall', None)
            print(f"  - imgsz={imgsz}: mAP@0.5-0.95={map_50_95}, Precision={precision}, Recall={recall}")
        else:
            print(f"  - imgsz={imgsz}: メトリクス取得不可")

else:
    print("❌ データセット設定ファイルが見つからないため、検証をスキップします。")


🔍 画像サイズ（32の倍数, 1500まで）で検証を実行しています...

--- 画像サイズ: 512 で検証中 ---
Ultralytics 8.3.203  Python-3.12.6 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
val: Fast image access  (ping: 0.20.1 ms, read: 27.95.9 MB/s, size: 179.6 KB)
val: Scanning \\?\D:\Data\pods\2025_yolo_pod\labels\YOLODataset\labels\val.cache... 13 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 13/13  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.1it/s 28.6s1:07


##　出力変換

In [ ]:
from ultralytics import YOLO

# Load a model

model = YOLO(r"D:\prog_py\Yolo_trainer\runs\train\yolo11s_640\weights\best.pt")  # load a custom trained model

# Export the model
model.export(format="onnx",
             name="yolo11s_640_onnx")

Ultralytics 8.3.203  Python-3.12.6 torch-2.6.0+cu124 CPU (Intel Core(TM) i7-10875H 2.30GHz)
YOLO11s-seg summary (fused): 113 layers, 10,067,203 parameters, 0 gradients, 32.8 GFLOPs

PyTorch: starting from 'D:\prog_py\Yolo_trainer\runs\train\yolo11s_640\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 37, 8400), (1, 32, 160, 160)) (19.7 MB)

ONNX: starting export with onnx 1.19.1 opset 19...
ONNX: slimming with onnxslim 0.1.72...
ONNX: export success  2.8s, saved as 'D:\prog_py\Yolo_trainer\runs\train\yolo11s_640\weights\best.onnx' (38.7 MB)

Export complete (3.4s)
Results saved to D:\prog_py\Yolo_trainer\runs\train\yolo11s_640\weights
Predict:         yolo predict task=segment model=D:\prog_py\Yolo_trainer\runs\train\yolo11s_640\weights\best.onnx imgsz=640  
Validate:        yolo val task=segment model=D:\prog_py\Yolo_trainer\runs\train\yolo11s_640\weights\best.onnx imgsz=640 data=D:\prog_py\Yolo_trainer\ultralytic_series\data\all_datasets\inner640_split

'D:\\prog_py\\Yolo_trainer\\runs\\train\\yolo11s_640\\weights\\best.onnx'

: 

## auto label

In [2]:
from ultralytics.data.annotator import auto_annotate

auto_annotate(data=r"Dataset\insta360\train",
              det_model=r"D:\prog_py\Yolo_trainer\runs\train\yolo11s_640\weights\best.pt",
              sam_model="sam2.1_l.pt")


image 1/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\024_01_b.jpg: 640x384 57 pods, 125.5ms
image 2/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\024_02_b.jpg: 640x288 63 pods, 78.8ms
image 3/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\024_02_f.jpg: 640x384 71 pods, 12.1ms
image 4/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\024_03_b.jpg: 640x384 55 pods, 14.3ms
image 5/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\024_03_f.jpg: 640x320 64 pods, 92.1ms
image 6/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\024_05_b.jpg: 640x320 29 pods, 10.4ms
image 7/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\024_05_f.jpg: 640x320 59 pods, 10.0ms
image 8/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\043_01_b.jpg: 640x416 66 pods, 90.7ms
image 9/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\043_01_f.jpg: 640x512 40 pods, 90.2ms
image 10/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\043_05_b.jpg: 640x352 39 pods, 76.9ms
image 11/69 d:\prog_py\yolo